In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
import json
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
actual_tags = []
with open ('/content/drive/MyDrive/Cleaned_Sentences_Task/tags_original.txt', 'r') as file:
    actual_tags = json.load(file)

In [ ]:
removed = []
with open ('/content/drive/MyDrive/Cleaned_Sentences_Task/sentences_without_tags.txt', 'r') as file:
    removed = json.load(file)

In [ ]:
print(len(removed))

331364


In [ ]:
def extract_features(sentence, index):
  return {
      'word':sentence[index],
      'is_first':index==0,
      'is_last':index ==len(sentence)-1,
      'prefix-1':sentence[index][0],
      'prefix-2':sentence[index][:2],
      'prefix-3':sentence[index][:3],
      'prefix-3':sentence[index][:4],
      'suffix-1':sentence[index][-1],
      'suffix-2':sentence[index][-2:],
      'suffix-3':sentence[index][-3:],
      'suffix-3':sentence[index][-4:],
      'prev_word':'' if index == 0 else sentence[index-1],
      'next_word':'' if index == 1 else sentence[index+1],
      'has_hyphen': '-' in sentence[index],
      'is_numeric': sentence[index].isdigit()
  }

In [ ]:
def transform_to_dataset(sentences):
  X, y = [], []
  for sents in sentences:
    for index in range(len(sents)):
      X.append(extract_features(sents, index))
  return X, actual_tags[0: 22000]

In [ ]:
X_, y_ = transform_to_dataset(removed[0: 11000])
for i in range(0, 100):
  print(X_[i], " ----------> ", y_[i])

{'word': 'आग', 'is_first': True, 'is_last': False, 'prefix-1': 'आ', 'prefix-2': 'आग', 'prefix-3': 'आग', 'suffix-1': 'ग', 'suffix-2': 'आग', 'suffix-3': 'आग', 'prev_word': '', 'next_word': 'की', 'has_hyphen': False, 'is_numeric': False}  ---------->  nn
{'word': 'की', 'is_first': False, 'is_last': True, 'prefix-1': 'क', 'prefix-2': 'की', 'prefix-3': 'की', 'suffix-1': 'ी', 'suffix-2': 'की', 'suffix-3': 'की', 'prev_word': 'आग', 'next_word': '', 'has_hyphen': False, 'is_numeric': False}  ---------->  psp
{'word': 'लिए', 'is_first': True, 'is_last': False, 'prefix-1': 'ल', 'prefix-2': 'लि', 'prefix-3': 'लिए', 'suffix-1': 'ए', 'suffix-2': 'िए', 'suffix-3': 'लिए', 'prev_word': '', 'next_word': 'सांडिआ', 'has_hyphen': False, 'is_numeric': False}  ---------->  psp
{'word': 'सांडिआ', 'is_first': False, 'is_last': True, 'prefix-1': 'स', 'prefix-2': 'सा', 'prefix-3': 'सांड', 'suffix-1': 'आ', 'suffix-2': 'िआ', 'suffix-3': 'ंडिआ', 'prev_word': 'लिए', 'next_word': '', 'has_hyphen': False, 'is_numeric'

In [ ]:
print(len(X_), len(y_))

22000 22000


In [ ]:
print(y_[0: 10])
print(actual_tags[0: 10])

['nn', 'psp', 'psp', 'nnpc', 'psp', 'nnpc', 'vaux', 'vaux', 'nnp', 'psp']
['nn', 'psp', 'psp', 'nnpc', 'psp', 'nnpc', 'vaux', 'vaux', 'nnp', 'psp']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_, y_, train_size=0.75)

In [ ]:
dict_vectorizer = DictVectorizer(sparse=False)
dict_vectorizer.fit(X_)
X_train = dict_vectorizer.transform(X_train)
X_test = dict_vectorizer.transform(X_test)

In [ ]:
X_ = dict_vectorizer.transform(X_)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_)
y_train = label_encoder.transform(y_train)
y_train_cat = to_categorical(y_train, num_classes=len(label_encoder.classes_))

In [ ]:
model_deep = Sequential()
model_deep.add(Dense(1024, activation='relu'))
model_deep.add(Dropout(0.2))
model_deep.add(BatchNormalization())
model_deep.add(Dense(512, activation='relu'))
model_deep.add(Dropout(0.2))
model_deep.add(Dense(256, activation='relu'))
model_deep.add(Dropout(0.2))
model_deep.add(BatchNormalization())
model_deep.add(Dense(128, activation='relu'))
model_deep.add(Dropout(0.2))
model_deep.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [ ]:
model_deep.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_deep.fit(X_train, y_train_cat, validation_split=0.2, epochs=50, batch_size=128)

Epoch 1/50
104/104 [==============================] - 31s 295ms/step - loss: 1.1723 - accuracy: 0.6736 - val_loss: 2.4328 - val_accuracy: 0.2330
Epoch 2/50
104/104 [==============================] - 30s 290ms/step - loss: 0.4240 - accuracy: 0.8755 - val_loss: 2.1762 - val_accuracy: 0.2752
Epoch 3/50
104/104 [==============================] - 30s 289ms/step - loss: 0.2520 - accuracy: 0.9228 - val_loss: 1.3639 - val_accuracy: 0.5721
Epoch 4/50
104/104 [==============================] - 30s 290ms/step - loss: 0.1799 - accuracy: 0.9430 - val_loss: 0.7519 - val_accuracy: 0.7815
Epoch 5/50
104/104 [==============================] - 30s 291ms/step - loss: 0.1457 - accuracy: 0.9536 - val_loss: 0.7128 - val_accuracy: 0.8121
Epoch 6/50
104/104 [==============================] - 30s 290ms/step - loss: 0.1210 - accuracy: 0.9601 - val_loss: 0.7814 - val_accuracy: 0.8200
Epoch 7/50
104/104 [==============================] - 30s 290ms/step - loss: 0.0995 - accuracy: 0.9689 - val_loss: 0.8483 - val_ac

In [ ]:
model_deep.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              22066176  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [ ]:
print("Accuracy Score = ", model_deep.evaluate(X_test, to_categorical(label_encoder.transform(y_test))))

180/180 [==============================] - 7s 39ms/step - loss: 1.0485 - accuracy: 0.8445
Accuracy Score =  [1.0485262870788574, 0.8445217609405518]


In [ ]:
predictions = list(model_deep.predict_classes(X_))
deep_tags = []
for i in predictions:
  deep_tags.extend(list(label_encoder.inverse_transform([i])))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
with open ('/content/drive/MyDrive/Cleaned_Sentences_Task/ann_predictions.txt', 'w+') as file:
    json.dump(deep_tags, file)